### Name: Mayur Jain
### Div: BE10-Q10
### Roll no: 43230
### Title: Assignment 5: Implement the Continuous Bag of Words (CBOW) Model

In [2]:
#importing libraries
from keras.preprocessing import text
from keras.utils import np_utils
from keras.preprocessing import sequence
from keras.utils import pad_sequences
import numpy as np
import pandas as pd

In [3]:
#taking random sentences as data
data = """Deep learning (also known as deep structured learning) is part of a broader family of machine learning methods based on artificial neural networks with representation learning. Learning can be supervised, semi-supervised or unsupervised. 
Deep-learning architectures such as deep neural networks, deep belief networks, deep reinforcement learning, recurrent neural networks, convolutional neural networks and Transformers have been applied to fields including computer vision, speech recognition, natural language processing, machine translation, bioinformatics, drug design, medical image analysis, climate science, material inspection and board game programs, where they have produced results comparable to and in some cases surpassing human expert performance.
"""
dl_data = data.split()

In [7]:
#tokenization
tokenizer = text.Tokenizer()
tokenizer.fit_on_texts(dl_data)
word2id = tokenizer.word_index

id2word = {v:k for k, v in word2id.items()}
wids = [[word2id[w] for w in text.text_to_word_sequence(doc)] for doc in dl_data]
print(wids)
vocab_size = len(word2id)
embed_size = 100
window_size = 2 

print('Vocabulary Size:', vocab_size)
print('Vocabulary Sample:', list(word2id.items())[:10])

[[2], [1], [12], [13], [6], [2], [14], [1], [15], [16], [7], [17], [18], [19], [7], [8], [1], [20], [21], [22], [23], [4], [3], [24], [25], [1], [1], [26], [27], [9], [28, 9], [29], [30], [2, 1], [31], [32], [6], [2], [4], [3], [2], [33], [3], [2], [34], [1], [35], [4], [3], [36], [4], [3], [5], [37], [10], [38], [39], [11], [40], [41], [42], [43], [44], [45], [46], [47], [48], [8], [49], [50], [51], [52], [53], [54], [55], [56], [57], [58], [59], [5], [60], [61], [62], [63], [64], [10], [65], [66], [67], [11], [5], [68], [69], [70], [71], [72], [73], [74]]
Vocabulary Size: 74
Vocabulary Sample: [('learning', 1), ('deep', 2), ('networks', 3), ('neural', 4), ('and', 5), ('as', 6), ('of', 7), ('machine', 8), ('supervised', 9), ('have', 10)]


In [4]:
#generating (context word, target/label word) pairs
def generate_context_word_pairs(corpus, window_size, vocab_size):
    context_length = window_size*2
    for words in corpus:
        sentence_length = len(words)
        for index, word in enumerate(words):
            context_words = []
            label_word   = []            
            start = index - window_size
            end = index + window_size + 1
            
            context_words.append([words[i] 
                                 for i in range(start, end) 
                                 if 0 <= i < sentence_length 
                                 and i != index])
            label_word.append(word)

            x = pad_sequences(context_words, maxlen=context_length)
            y = np_utils.to_categorical(label_word, vocab_size)
            yield (x, y)
            
i = 0
for x, y in generate_context_word_pairs(corpus=wids, window_size=window_size, vocab_size=vocab_size):
    if 0 not in x[0]:
        # print('Context (X):', [id2word[w] for w in x[0]], '-> Target (Y):', id2word[np.argwhere(y[0])[0][0]])
    
        if i == 10:
            break
        i += 1

In [5]:
#model building
import keras.backend as K
from keras.models import Sequential
from keras.layers import Dense, Embedding, Lambda

cbow = Sequential()
cbow.add(Embedding(input_dim=vocab_size, output_dim=embed_size, input_length=window_size*2))
cbow.add(Lambda(lambda x: K.mean(x, axis=1), output_shape=(embed_size,)))
cbow.add(Dense(vocab_size, activation='softmax'))
cbow.compile(loss='categorical_crossentropy', optimizer='rmsprop')

print(cbow.summary())

# from IPython.display import SVG
# from keras.utils.vis_utils import model_to_dot

# SVG(model_to_dot(cbow, show_shapes=True, show_layer_names=False, rankdir='TB').create(prog='dot', format='svg'))

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 4, 100)            7500      
                                                                 
 lambda (Lambda)             (None, 100)               0         
                                                                 
 dense (Dense)               (None, 75)                7575      
                                                                 
Total params: 15,075
Trainable params: 15,075
Non-trainable params: 0
_________________________________________________________________
None


In [6]:
for epoch in range(1, 6):
    loss = 0.
    i = 0
    for x, y in generate_context_word_pairs(corpus=wids, window_size=window_size, vocab_size=vocab_size):
        i += 1
        loss += cbow.train_on_batch(x, y)
        if i % 100000 == 0:
            print('Processed {} (context, word) pairs'.format(i))

    print('Epoch:', epoch, '\tLoss:', loss)
    print()

Epoch: 1 	Loss: 433.71598768234253

Epoch: 2 	Loss: 428.86942744255066

Epoch: 3 	Loss: 425.42393493652344

Epoch: 4 	Loss: 422.18471336364746

Epoch: 5 	Loss: 419.7406575679779



In [7]:
weights = cbow.get_weights()[0]
weights = weights[1:]
print(weights.shape)

pd.DataFrame(weights, index=list(id2word.values())[1:]).head()

(74, 100)


0         1         2         3         4         5   \
deep     -0.004552  0.015162  0.019465 -0.027191  0.043231 -0.031147   
networks  0.035097 -0.024231 -0.017355  0.004131  0.027114 -0.016241   
neural   -0.032488 -0.007066 -0.019217 -0.007592  0.019406 -0.027018   
and      -0.011250 -0.030099  0.030994  0.007380 -0.038013  0.044312   
as       -0.006021  0.016620 -0.011210  0.025741  0.030643 -0.020005   

                6         7         8         9   ...        90        91  \
deep      0.028567  0.009478  0.054867  0.059350  ...  0.018952  0.032517   
networks  0.018686 -0.034545  0.040944 -0.017245  ...  0.002348 -0.020592   
neural   -0.024834  0.017910  0.011590 -0.036070  ...  0.034484 -0.012461   
and      -0.025339 -0.034315 -0.008518 -0.005537  ...  0.001177  0.038307   
as        0.017006  0.003311 -0.030919  0.003577  ... -0.008984  0.009878   

                92        93        94        95        96        97  \
deep      0.017693 -0.043570 -0.012374 -0.035991  0.042621 -0.001871   
networks  0.011543  0.002631  0.045674 -0.004343  0.001119  0.004260   
neural   -0.044156 -0.025378  0.005906  0.042814 -0.005753 -0.007971   
and       0.033360 -0.013410  0.027830  0.035182 -0.019216  0.045571   
as       -0.013783  0.049174 -0.036771  0.043503 -0.008915  0.004619   

                98        99  
deep     -0.006317  0.007518  
networks -0.005308  0.033788  
neural    0.016142  0.045053  
and      -0.009555  0.048420  
as        0.026007 -0.035560  

[5 rows x 100 columns]

In [8]:
from sklearn.metrics.pairwise import euclidean_distances

distance_matrix = euclidean_distances(weights)
print(distance_matrix.shape)

similar_words = {search_term: [id2word[idx] for idx in distance_matrix[word2id[search_term]-1].argsort()[1:6]+1] 
                   for search_term in ['deep']}

similar_words

(74, 74)


{'deep': ['have', 'applied', 'game', 'been', 'structured']}